こういう感じのコメントを付ける

```
constexpr double LINE_LENGTH_LOSS_STEPPING_PROGRESS_RATE = 0.7001976706415303;  // OPTIMIZE [0.0, 1.0]
constexpr double  LINE_LENGTH_LOSS_WEIGHT = 8.009299591633644e-06;  // OPTIMIZE [1e-8, 1e-4] LOG
constexpr int N_RANDOM_FACE_CHOICE = 3;  // OPTIMIZE [1, 10]
std::string hoge="hoge";//OPTIMIZE{"11", "unnn" ,"ooo"}
```

`[` と `,` の間の文字列が Python の式として評価され、その結果の型が最適化に使われる

/* でのコメントアウトは効かないので注意

In [1]:
import re
import sys
from time import sleep
from pprint import pprint
from functools import partial

import optuna
try:
    import pandas as pd
except:
    pass


FILENAME = "answer.cpp"

In [2]:
!mkdir optimize_tmp

mkdir: cannot create directory ‘optimize_tmp’: File exists


In [3]:
# 元のファイルのコンパイル
!g++ {FILENAME} -std=gnu++17 -Wall -Wextra -O2 -DONLINE_JUDGE -o {FILENAME}.out

answer.cpp: In constructor ‘State::State(const Input&)’:
answer.cpp:393:30: warning: ‘State::dfs_path’ will be initialized after [-Wreorder]
  393 |  Stack<Direction, MAX_DEPTH> dfs_path;
      |                              ^~~~~~~~
answer.cpp:391:22: warning:   ‘Stack<int, 2500> State::closed’ [-Wreorder]
  391 |  Stack<int, 50 * 50> closed;
      |                      ^~~~~~
answer.cpp:401:2: warning:   when initialized here [-Wreorder]
  401 |  State(const Input& a_input) : input(a_input), path(), dfs_path(), closed(),
      |  ^~~~~


In [6]:
# テストケース生成
!wget https://img.atcoder.jp/ahc002/c993bb7f09d9f8857fc90951fc6af11d.zip
!unzip c993bb7f09d9f8857fc90951fc6af11d.zip

SEED = 201
N_TESTCASES = 50
!seq {SEED} {SEED+N_TESTCASES-1} | cargo run --release --manifest-path ./tools/Cargo.toml --bin gen

   Compiling libc v0.2.93
   Compiling getrandom v0.1.16
   Compiling proc-macro2 v0.4.30
   Compiling cfg-if v1.0.0
   Compiling unicode-xid v0.1.0
   Compiling syn v0.15.44
   Compiling ppv-lite86 v0.2.10
   Compiling either v1.6.1
   Compiling lazy_static v1.4.0
   Compiling svg v0.9.1
   Compiling itertools v0.9.0          ] 3/28: getrandom(build.rs), ppv-...
   Compiling quote v0.6.13            ] 13/28: libc, itertools, svg, pro...
   Compiling rand_core v0.5.1         ] 18/28: getrandom, syn              
   Compiling rand_pcg v0.2.1>         ] 19/28: syn, rand_core              
   Compiling rand_chacha v0.2.2
   Compiling rand v0.7.3======>       ] 21/28: syn, rand_chacha            
   Compiling proconio-derive v0.1.9   ] 23/28: syn                         
   Compiling proconio v0.3.6=====>    ] 24/28: proconio-derive             
   Compiling tools v1.0.0 (/home/nagis/ahc3/tools)conio                    
    Finished release [optimized] target(s) in 11.72sin)               

In [7]:
!tools/target/release/gen tools/seeds.txt

In [8]:
!ls

answer.cpp	c993bb7f09d9f8857fc90951fc6af11d.zip  optimize_tmp     tools
answer.cpp.out	in				      optimizer.ipynb


In [8]:
# ここでエラー出力は何かおかしいかもしれない


# パラメータ抽出
with open(FILENAME) as f:
    answer = f.read()


variables_optimize = []

for left, value, right, options in re.findall(r"^([^/\n]*=\s*)(.+?)(\s*;\s*//(?:.*\W)?OPTIMIZE(\W.*))$", answer, re.MULTILINE):
    name = left.replace("=", "").strip().split()[-1]
    
    searched = re.search(r".*\[(?P<min>.*),(?P<max>.*)\].*", options)
    if searched:
        min_value = max_value = None
        try:
            min_value = eval(searched.group("min"))
            max_value = eval(searched.group("max"))
            assert min_value <= max_value
        except Exception as e:
            print(f"searched={searched}", file=sys.stderr)
            print(e, file=sys.stderr)
            continue
        log = "LOG" in options  # 雑、直したほうが良い
        if type(min_value) != type(max_value):
            print(f"searched={searched}", file=sys.stderr)
            print("types not matched", file=sys.stderr)
            continue
        if isinstance(min_value, int):
            method = "suggest_int"
        elif isinstance(min_value, float):
            method = "suggest_float"
        else:
            print(f"searched={searched}", file=sys.stderr)
            print(f"unknown type ({type(min_value)})", file=sys.stderr)
            continue
        variables_optimize.append({
            "name": name,
            "method": method,
            "min": min_value,
            "max": max_value,
            "log": log,
            "left": left,
            "right": right,
        })
    elif searched := re.search(r".*\{(?P<choices>.*?)\}.*", options):
        choices = list(map(lambda x: x.strip(), searched.group("choices").split(",")))
        variables_optimize.append({
            "name": name,
            "method": "suggest_categorical",
            "choices": choices,
            "left": left,
            "right": right,
        })
    else:
        print(f"searched={searched}", file=sys.stderr)
        print(f"pattern was matched but options are incorrect.", file=sys.stderr)

print(len(variables_optimize), "variables were found.")
if globals().get("pd"):
    display(pd.DataFrame(variables_optimize))
else:
    pprint(variables_optimize)

5 variables were found.


,name,method,min,max,log,left,right
0,COEF_CENTER_L1,suggest_float,-100.0,100.0,False,\t\tconstexpr double COEF_CENTER_L1 =,"; // OPTIMIZE [-100.0, 100.0]"
1,COEF_CENTER_L1_SQ,suggest_float,0.0,200.0,False,\t\tconstexpr double COEF_CENTER_L1_SQ =,"; // OPTIMIZE [0.0, 200.0]"
2,COEF_POINT,suggest_float,0.0,100.0,False,\t\tconstexpr double COEF_POINT =,"; // OPTIMIZE [0.0, 100.0]"
3,COEF_DIST,suggest_float,0.0,10000.0,False,\t\tconstexpr double COEF_DIST =,"; // OPTIMIZE [0.0, 10000.0]"
4,COEF_TWIST,suggest_float,0.0,1000000.0,False,\t\tconstexpr double COEF_TWIST =,"; // OPTIMIZE [0.0, 1000000.0]"


In [9]:
def escape(string):  # 正規表現の中でそのまま使いたい文字列をエスケープ
    res = !echo '{string}' | sed -e 's/[]\/$*.^[]/\\&/g'
    return res[0]

def escape_sed(string):  # sed の置換後の文字列用のエスケープ
    res = !echo '{string}' | sed -e 's/[\/&]/\\&/g'
    return res[0]


def objective_with_index(trial, n_internal_parallel):
    index_parallel = trial.number % 100
    print(f"{index_parallel=}")
    
    parameters_changed_filename = f"{index_parallel}_{FILENAME}"
    directory_input = "./in"  # "/" をつけない
    
    !mkdir {index_parallel}_out
    !mkdir {index_parallel}_score
    !rm {index_parallel}_out/*
    !rm {index_parallel}_score/*
    
    # ファイル作成
    !cp {FILENAME} {parameters_changed_filename}
    sed_options = [f"-i {parameters_changed_filename}"]
    for variable in variables_optimize:
        if variable["method"] == "suggest_categorical":
            val =  trial.suggest_categorical(variable["name"], variable["choices"])
        else:
            val = getattr(trial, variable["method"])(variable["name"], variable["min"], variable["max"])
        left = variable["left"]
        right = variable["right"]
        sed_options.append(f"""-e 's/^{escape(left)}.*{escape(right)}$/{escape_sed(left)}{val}{escape_sed(right)}/'""")
    command_sed = f"sed {' '.join(sed_options)}"
    print(command_sed)
    !{command_sed}
    
    # コンパイル
    command_compile = f"g++ {parameters_changed_filename} -std=gnu++17 -O2 -DONLINE_JUDGE -o {parameters_changed_filename}.out"
    print(command_compile)
    !{command_compile}
    # 実行・採点コマンド (@ はファイル名)
    command_exec = (
        f"./{parameters_changed_filename}.out < {directory_input}/@ > ./{index_parallel}_out/@;"
        f"cargo run --release --manifest-path ./tools/Cargo.toml --bin vis {directory_input}/@ ./{index_parallel}_out/@ 2> /dev/null | sed -e s/[^0-9]//g > ./{index_parallel}_score/@;"
    )
    # 並列実行 (sed はパスのディレクトリ部分を消してファイル名にしてる)
    !find {directory_input}/* | sed 's!^.*/!!' | xargs -I@ -P {n_internal_parallel} sh -c '{command_exec}'
    
    # 集計
    file_paths = !find ./{index_parallel}_score/*
    sum_score = 0
    for file_path in file_paths:  
        with open(file_path) as f:
            sum_score += int(f.read())
    return sum_score



In [10]:
variables_optimize

[{'name': 'COEF_CENTER_L1',
  'method': 'suggest_float',
  'min': -100.0,
  'max': 100.0,
  'log': False,
  'left': '\t\tconstexpr double COEF_CENTER_L1 = ',
  'right': ';  // OPTIMIZE [-100.0, 100.0]'},
 {'name': 'COEF_CENTER_L1_SQ',
  'method': 'suggest_float',
  'min': 0.0,
  'max': 200.0,
  'log': False,
  'left': '\t\tconstexpr double COEF_CENTER_L1_SQ = ',
  'right': ';  // OPTIMIZE [0.0, 200.0]'},
 {'name': 'COEF_POINT',
  'method': 'suggest_float',
  'min': 0.0,
  'max': 100.0,
  'log': False,
  'left': '\t\tconstexpr double COEF_POINT = ',
  'right': ';  // OPTIMIZE [0.0, 100.0]'},
 {'name': 'COEF_DIST',
  'method': 'suggest_float',
  'min': 0.0,
  'max': 10000.0,
  'log': False,
  'left': '\t\tconstexpr double COEF_DIST = ',
  'right': ';  // OPTIMIZE [0.0, 10000.0]'},
 {'name': 'COEF_TWIST',
  'method': 'suggest_float',
  'min': 0.0,
  'max': 1000000.0,
  'log': False,
  'left': '\t\tconstexpr double COEF_TWIST = ',
  'right': ';  // OPTIMIZE [0.0, 1000000.0]'}]

In [12]:
variables_optimize[0]["max"] = 100000.0

In [14]:
storage_path = f"study.db"
storage = f"sqlite:///{storage_path}"
study_name = "study"
study = optuna.create_study(storage=storage, load_if_exists=True, study_name=study_name, direction="maximize")

def callback(study, trial):
    try:
        if study.best_value == trial.value:  ############# 要変更
            print(f"Updated! {study.best_value} -> {trial.value}")
            index_parallel = trial.number % 100
            parameters_changed_filename = f"{index_parallel}_{FILENAME}"
            !cp {parameters_changed_filename} 00000optimized.cpp
        print("callback ok")
    except:
        print(":(")

objective = partial(objective_with_index, n_internal_parallel=4)
study.optimize(objective, n_trials=3000, timeout=3600, n_jobs=6, callbacks=[callback])

[I 2021-04-25 15:50:02,667] Using an existing study with name 'study' instead of creating a new one.


index_parallel=31
index_parallel=27
index_parallel=28
index_parallel=30
index_parallel=29
mkdir: cannot create directory ‘31_out’: File exists
mkdir: cannot create directory ‘27_out’: File exists
index_parallel=32
mkdir: cannot create directory ‘28_out’: File exists
mkdir: cannot create directory ‘30_out’: File exists
mkdir: cannot create directory ‘29_out’: File exists
mkdir: cannot create directory ‘32_out’: File exists
mkdir: cannot create directory ‘27_score’: File exists
mkdir: cannot create directory ‘28_score’: File exists
mkdir: cannot create directory ‘31_score’: File exists
mkdir: cannot create directory ‘30_score’: File exists
mkdir: cannot create directory ‘32_score’: File exists
mkdir: cannot create directory ‘29_score’: File exists
sed -i 27_answer.cpp -e 's/^		constexpr double COEF_CENTER_L1 = .*;  \/\/ OPTIMIZE \[-100\.0, 100\.0\]$/		constexpr double COEF_CENTER_L1 = 10.847007590217345;  \/\/ OPTIMIZE [-100.0, 100.0]/' -e 's/^		constexpr double COEF_CENTER_L1_SQ = .*;  

[I 2021-04-25 15:50:25,400] Trial 832 finished with value: 938248.0 and parameters: {'COEF_CENTER_L1': 37004.57201587735, 'COEF_CENTER_L1_SQ': 0.32092144803301226, 'COEF_POINT': 95.99867651499586, 'COEF_DIST': 180.15663550698724, 'COEF_TWIST': 711329.833896799}. Best is trial 431 with value: 1924103.0.


callback ok
index_parallel=33
mkdir: cannot create directory ‘33_out’: File exists
mkdir: cannot create directory ‘33_score’: File exists
sed -i 33_answer.cpp -e 's/^		constexpr double COEF_CENTER_L1 = .*;  \/\/ OPTIMIZE \[-100\.0, 100\.0\]$/		constexpr double COEF_CENTER_L1 = 1484.7979555958473;  \/\/ OPTIMIZE [-100.0, 100.0]/' -e 's/^		constexpr double COEF_CENTER_L1_SQ = .*;  \/\/ OPTIMIZE \[0\.0, 200\.0\]$/		constexpr double COEF_CENTER_L1_SQ = 9.421119654587013;  \/\/ OPTIMIZE [0.0, 200.0]/' -e 's/^		constexpr double COEF_POINT = .*;  \/\/ OPTIMIZE \[0\.0, 100\.0\]$/		constexpr double COEF_POINT = 96.07985647054926;  \/\/ OPTIMIZE [0.0, 100.0]/' -e 's/^		constexpr double COEF_DIST = .*;  \/\/ OPTIMIZE \[0\.0, 10000\.0\]$/		constexpr double COEF_DIST = 11.203002646992516;  \/\/ OPTIMIZE [0.0, 10000.0]/' -e 's/^		constexpr double COEF_TWIST = .*;  \/\/ OPTIMIZE \[0\.0, 1000000\.0\]$/		constexpr double COEF_TWIST = 731591.8866672533;  \/\/ OPTIMIZE [0.0, 1000000.0]/'
g++ 33_answer.cp

[I 2021-04-25 15:50:39,454] Trial 827 finished with value: 1427962.0 and parameters: {'COEF_CENTER_L1': 10.847007590217345, 'COEF_CENTER_L1_SQ': 0.46217459095208646, 'COEF_POINT': 95.98407932509232, 'COEF_DIST': 1.405529489607689, 'COEF_TWIST': 561935.3223297969}. Best is trial 431 with value: 1924103.0.


...

[I 2021-04-25 15:56:42,987] Trial 895 finished with value: 1732966.0 and parameters: {'COEF_CENTER_L1': 3954.1548326286365, 'COEF_CENTER_L1_SQ': 11.436038570751801, 'COEF_POINT': 90.03017095152882, 'COEF_DIST': 351.71485874185106, 'COEF_TWIST': 712350.6014326484}. Best is trial 431 with value: 1924103.0.


callback ok


[I 2021-04-25 15:56:45,304] Trial 896 finished with value: 1786482.0 and parameters: {'COEF_CENTER_L1': 3858.0515659668786, 'COEF_CENTER_L1_SQ': 7.732013364647763, 'COEF_POINT': 89.38279338732791, 'COEF_DIST': 321.21191433664427, 'COEF_TWIST': 760505.0635003906}. Best is trial 431 with value: 1924103.0.


callback ok


KeyboardInterrupt: 

In [ ]:
# $ apt install sysstat
# $ mpstat -P ALL 5
# とかやると CPU 負荷を見られる

In [19]:
print(f"{study.best_value}")
pprint(study.best_params)

48621734068.0
{'ANNEALING_A': -9.663753766636084,
 'ANNEALING_B': 0.23708439451937813,
 'ANNEALING_END': 0.004139552269746777,
 'ANNEALING_START': 0.4830473145504588,
 'GRADIENT_CLIPPING': 0.00019811172822438805,
 'GRADIENT_DESCENT_LR': 10380014.948175553,
 'GRADIENT_DESCENT_MOMENTUM': 0.3276696959783839,
 'GRADIENT_DESCENT_STEPS': 30,
 'GRADIENT_DESCENT_STEPS_STEPPING_PROGRESS_RATE': 0.9031609842262478,
 'LINE_LENGTH_LOSS_STEPPING_PROGRESS_RATE': 0.021698592868210798,
 'LINE_LENGTH_LOSS_WEIGHT': 9.614201554394316e-07,
 'N_RANDOM_FACE_CHOICE': 8,
 'hoge': '"ooo"'}
